In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
from scipy.optimize import minimize
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

def read_csv(csv_path):
    
    data = np.genfromtxt(csv_path, delimiter=',', skip_header=1)  
    paths = []
    unique_paths = np.unique(data[:, 0])
    
    for path_id in unique_paths:
        path_data = data[data[:, 0] == path_id][:, 1:]
        paths.append(path_data)
    
    return paths

def find_symmetry_axes(points):
   
    
    pca = PCA(n_components=2)
    pca.fit(points)
    axes = pca.components_

   
    def inertia(axis):
        
        projections = np.dot(points, axis)
        mean_projection = np.mean(projections)
        inertia = np.sum((projections - mean_projection) ** 2)
        return inertia
    
    def objective(angle):
        axis = np.array([np.cos(angle), np.sin(angle)])
        return inertia(axis)
    
    angles = np.linspace(0, 2 * np.pi, 360)
    results = [objective(angle) for angle in angles]
    min_angle = angles[np.argmin(results)]
    min_axis = np.array([np.cos(min_angle), np.sin(min_angle)])
    
    return min_axis, min_angle

def check_symmetry(points, axis):
    
    def reflect_point(point, axis):
        
        axis_normal = np.array([-axis[1], axis[0]])
        projection = np.dot(point, axis_normal)
        return point - 2 * projection * axis_normal
    
    reflected_points = np.array([reflect_point(p, axis) for p in points])
    distances = np.linalg.norm(reflected_points[:, np.newaxis] - points, axis=2)
    max_distance = np.max(distances)
    
    return max_distance < 1e-6  

def plot_symmetry_detection(points, axis, title='Symmetry Detection'):
    
    plt.figure(figsize=(8, 8))
    plt.scatter(points[:, 0], points[:, 1], label='Points')
    plt.quiver(0, 0, axis[0], axis[1], angles='xy', scale_units='xy', scale=1, color='r', label='Symmetry Axis')
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.grid(True)
    plt.title(title)
    plt.legend()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

def detect_symmetry_in_curves(csv_path):
    
    curves = read_csv(csv_path)
    symmetry_results = []

    for curve in curves:
        if len(curve) < 3:
            continue
        
        points = np.array(curve)
        
        
        axis, angle = find_symmetry_axes(points)
        
        
        if check_symmetry(points, axis):
            symmetry_results.append({
                'axis': axis,
                'angle': angle
            })
            plot_symmetry_detection(points, axis, title='Detected Symmetry Axis')

    return symmetry_results


csv_file_path = 'problems/frag2.csv'
symmetries = detect_symmetry_in_curves(csv_file_path)
print(f"Detected Symmetries: {symmetries}")
